In [1]:
import pandas as pd
import pickle
import numpy as np

In [3]:
#we will open level-2 file
#f = open("dataset_lev2.pkl", mode="r", encoding="utf-8")
df = pickle.load(open('dataset_lev2.pkl','rb'))

In [4]:
df

# batting_team
# bowling team
# city
# current score
# ball left
# wickets left
# current rr
# last five

,match_id,batting_team,bowling_team,ball,runs,player_dismissed,city,venue
0,2,Australia,Pakistan,0.1,0,0,Brisbane,"Brisbane Cricket Ground, Woolloongabba"
1,2,Australia,Pakistan,0.2,0,0,Brisbane,"Brisbane Cricket Ground, Woolloongabba"
2,2,Australia,Pakistan,0.3,0,0,Brisbane,"Brisbane Cricket Ground, Woolloongabba"
3,2,Australia,Pakistan,0.4,0,0,Brisbane,"Brisbane Cricket Ground, Woolloongabba"
4,2,Australia,Pakistan,0.5,1,0,Brisbane,"Brisbane Cricket Ground, Woolloongabba"
...,...,...,...,...,...,...,...,...
244,1795,Sri Lanka,Australia,39.5,0,0,NaN,Pallekele International Cricket Stadium
245,1795,Sri Lanka,Australia,39.6,0,0,NaN,Pallekele International Cricket Stadium
246,1795,Sri Lanka,Australia,39.7,1,0,NaN,Pallekele International Cricket Stadium
247,1795,Sri Lanka,Australia,40.1,0,0,NaN,Pallekele International Cricket Stadium


In [5]:
df.isnull().sum()

match_id                0
batting_team            0
bowling_team            0
ball                    0
runs                    0
player_dismissed        0
city                81124
venue                   0
dtype: int64

In [6]:
#in 81124 rows values are missing. In those rows value of cities are missing
df['city'].isnull()

0      False
1      False
2      False
3      False
4      False
       ...  
244     True
245     True
246     True
247     True
248     True
Name: city, Length: 436156, dtype: bool

In [7]:
#to solve problem we will use venue
df[df['city'].isnull()]['venue'].value_counts()

Harare Sports Club                         16957
Sydney Cricket Ground                      10217
Rangiri Dambulla International Stadium     10051
Melbourne Cricket Ground                    8696
Adelaide Oval                               7725
Sharjah Cricket Stadium                     7461
Pallekele International Cricket Stadium     6721
Dubai International Cricket Stadium         6314
Rawalpindi Cricket Stadium                  1513
Multan Cricket Stadium                      1509
Queenstown Events Centre                     909
Chittagong Divisional Stadium                900
Perth Stadium                                526
Dubai Sports City Cricket Stadium            518
Galle International Stadium                  513
Sharjah Cricket Association Stadium          303
Sheikhupura Stadium                          291
Name: venue, dtype: int64

In [8]:
#if the city value is empty, then it will be replaced by the 1st letter of venue
cities = np.where(df['city'].isnull(),df['venue'].str.split().apply(lambda x:x[0]),df['city'])

In [9]:
df['city'] = cities

In [10]:
df.isnull().sum()

match_id            0
batting_team        0
bowling_team        0
ball                0
runs                0
player_dismissed    0
city                0
venue               0
dtype: int64

In [11]:
#then drop the venue
df.drop(columns=['venue'],inplace=True)

In [12]:
df

,match_id,batting_team,bowling_team,ball,runs,player_dismissed,city
0,2,Australia,Pakistan,0.1,0,0,Brisbane
1,2,Australia,Pakistan,0.2,0,0,Brisbane
2,2,Australia,Pakistan,0.3,0,0,Brisbane
3,2,Australia,Pakistan,0.4,0,0,Brisbane
4,2,Australia,Pakistan,0.5,1,0,Brisbane
...,...,...,...,...,...,...,...
244,1795,Sri Lanka,Australia,39.5,0,0,Pallekele
245,1795,Sri Lanka,Australia,39.6,0,0,Pallekele
246,1795,Sri Lanka,Australia,39.7,1,0,Pallekele
247,1795,Sri Lanka,Australia,40.1,0,0,Pallekele


In [13]:
df['city'].value_counts()

Mirpur                25040
London                18101
Colombo               17888
Harare                16957
Bulawayo              11016
                      ...  
Jamshedpur              300
Sheikhupura             291
Thiruvananthapuram      198
Gros Islet              179
Providence               80
Name: city, Length: 114, dtype: int64

In [14]:
#there are 114 cities, but we will not consider all. we will consider where minimum 5 matches occur.
eligible_cities = df['city'].value_counts()[df['city'].value_counts() > 1500].index.tolist()

In [15]:
#now we will keep the data of eligable cities
df = df[df['city'].isin(eligible_cities)]

In [16]:
df

,match_id,batting_team,bowling_team,ball,runs,player_dismissed,city
0,2,Australia,Pakistan,0.1,0,0,Brisbane
1,2,Australia,Pakistan,0.2,0,0,Brisbane
2,2,Australia,Pakistan,0.3,0,0,Brisbane
3,2,Australia,Pakistan,0.4,0,0,Brisbane
4,2,Australia,Pakistan,0.5,1,0,Brisbane
...,...,...,...,...,...,...,...
244,1795,Sri Lanka,Australia,39.5,0,0,Pallekele
245,1795,Sri Lanka,Australia,39.6,0,0,Pallekele
246,1795,Sri Lanka,Australia,39.7,1,0,Pallekele
247,1795,Sri Lanka,Australia,40.1,0,0,Pallekele


In [17]:
#now we will focus on current score 
df.groupby('match_id').cumsum()['runs']

0        0
1        0
2        0
3        0
4        1
      ... 
244    194
245    194
246    195
247    195
248    195
Name: runs, Length: 405793, dtype: int64

In [18]:
#then we will put it in new column named current score
df['current_score'] = df.groupby('match_id').cumsum()['runs']

C:\Users\User\AppData\Local\Temp\ipykernel_11140\4127419751.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['current_score'] = df.groupby('match_id').cumsum()['runs']


In [19]:
df

,match_id,batting_team,bowling_team,ball,runs,player_dismissed,city,current_score
0,2,Australia,Pakistan,0.1,0,0,Brisbane,0
1,2,Australia,Pakistan,0.2,0,0,Brisbane,0
2,2,Australia,Pakistan,0.3,0,0,Brisbane,0
3,2,Australia,Pakistan,0.4,0,0,Brisbane,0
4,2,Australia,Pakistan,0.5,1,0,Brisbane,1
...,...,...,...,...,...,...,...,...
244,1795,Sri Lanka,Australia,39.5,0,0,Pallekele,194
245,1795,Sri Lanka,Australia,39.6,0,0,Pallekele,194
246,1795,Sri Lanka,Australia,39.7,1,0,Pallekele,195
247,1795,Sri Lanka,Australia,40.1,0,0,Pallekele,195


In [20]:
#then we will work on ball column. 0.1 means over=0 and ball=1. so we will seperate over and ball column
df['over'] = df['ball'].apply(lambda x:str(x).split(".")[0])
df['ball_no'] = df['ball'].apply(lambda x:str(x).split(".")[1])

C:\Users\User\AppData\Local\Temp\ipykernel_11140\2015637041.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['over'] = df['ball'].apply(lambda x:str(x).split(".")[0])
C:\Users\User\AppData\Local\Temp\ipykernel_11140\2015637041.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['ball_no'] = df['ball'].apply(lambda x:str(x).split(".")[1])


In [21]:
df

,match_id,batting_team,bowling_team,ball,runs,player_dismissed,city,current_score,over,ball_no
0,2,Australia,Pakistan,0.1,0,0,Brisbane,0,0,1
1,2,Australia,Pakistan,0.2,0,0,Brisbane,0,0,2
2,2,Australia,Pakistan,0.3,0,0,Brisbane,0,0,3
3,2,Australia,Pakistan,0.4,0,0,Brisbane,0,0,4
4,2,Australia,Pakistan,0.5,1,0,Brisbane,1,0,5
...,...,...,...,...,...,...,...,...,...,...
244,1795,Sri Lanka,Australia,39.5,0,0,Pallekele,194,39,5
245,1795,Sri Lanka,Australia,39.6,0,0,Pallekele,194,39,6
246,1795,Sri Lanka,Australia,39.7,1,0,Pallekele,195,39,7
247,1795,Sri Lanka,Australia,40.1,0,0,Pallekele,195,40,1


In [22]:
#then calculate bowled balls
df['balls_bowled'] = (df['over'].astype('int')*6) + df['ball_no'].astype('int')

C:\Users\User\AppData\Local\Temp\ipykernel_11140\1276871035.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['balls_bowled'] = (df['over'].astype('int')*6) + df['ball_no'].astype('int')


In [23]:
df

,match_id,batting_team,bowling_team,ball,runs,player_dismissed,city,current_score,over,ball_no,balls_bowled
0,2,Australia,Pakistan,0.1,0,0,Brisbane,0,0,1,1
1,2,Australia,Pakistan,0.2,0,0,Brisbane,0,0,2,2
2,2,Australia,Pakistan,0.3,0,0,Brisbane,0,0,3,3
3,2,Australia,Pakistan,0.4,0,0,Brisbane,0,0,4,4
4,2,Australia,Pakistan,0.5,1,0,Brisbane,1,0,5,5
...,...,...,...,...,...,...,...,...,...,...,...
244,1795,Sri Lanka,Australia,39.5,0,0,Pallekele,194,39,5,239
245,1795,Sri Lanka,Australia,39.6,0,0,Pallekele,194,39,6,240
246,1795,Sri Lanka,Australia,39.7,1,0,Pallekele,195,39,7,241
247,1795,Sri Lanka,Australia,40.1,0,0,Pallekele,195,40,1,241


In [24]:
#calculate how many balls are left
df['balls_left'] = 300 - df['balls_bowled']
df['balls_left'] = df['balls_left'].apply(lambda x:0 if x<0 else x)

C:\Users\User\AppData\Local\Temp\ipykernel_11140\2616794747.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['balls_left'] = 300 - df['balls_bowled']
C:\Users\User\AppData\Local\Temp\ipykernel_11140\2616794747.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['balls_left'] = df['balls_left'].apply(lambda x:0 if x<0 else x)


In [25]:
df

,match_id,batting_team,bowling_team,ball,runs,player_dismissed,city,current_score,over,ball_no,balls_bowled,balls_left
0,2,Australia,Pakistan,0.1,0,0,Brisbane,0,0,1,1,299
1,2,Australia,Pakistan,0.2,0,0,Brisbane,0,0,2,2,298
2,2,Australia,Pakistan,0.3,0,0,Brisbane,0,0,3,3,297
3,2,Australia,Pakistan,0.4,0,0,Brisbane,0,0,4,4,296
4,2,Australia,Pakistan,0.5,1,0,Brisbane,1,0,5,5,295
...,...,...,...,...,...,...,...,...,...,...,...,...
244,1795,Sri Lanka,Australia,39.5,0,0,Pallekele,194,39,5,239,61
245,1795,Sri Lanka,Australia,39.6,0,0,Pallekele,194,39,6,240,60
246,1795,Sri Lanka,Australia,39.7,1,0,Pallekele,195,39,7,241,59
247,1795,Sri Lanka,Australia,40.1,0,0,Pallekele,195,40,1,241,59


In [26]:
#then work on wicket left. if the playes_dismissed value = 0, we will keep 0 else 1. then calculate cumulative sum. then subtract from 10
df['player_dismissed'] = df['player_dismissed'].apply(lambda x:0 if x=='0' else 1)
df['player_dismissed'] = df['player_dismissed'].astype('int')
df['player_dismissed'] = df.groupby('match_id').cumsum()['player_dismissed']
df['wickets_left'] = 10 - df['player_dismissed']

C:\Users\User\AppData\Local\Temp\ipykernel_11140\4088362777.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['player_dismissed'] = df['player_dismissed'].apply(lambda x:0 if x=='0' else 1)
C:\Users\User\AppData\Local\Temp\ipykernel_11140\4088362777.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['player_dismissed'] = df['player_dismissed'].astype('int')
C:\Users\User\AppData\Local\Temp\ipykernel_11140\4088362777.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice fro

In [27]:
df

,match_id,batting_team,bowling_team,ball,runs,player_dismissed,city,current_score,over,ball_no,balls_bowled,balls_left,wickets_left
0,2,Australia,Pakistan,0.1,0,0,Brisbane,0,0,1,1,299,10
1,2,Australia,Pakistan,0.2,0,0,Brisbane,0,0,2,2,298,10
2,2,Australia,Pakistan,0.3,0,0,Brisbane,0,0,3,3,297,10
3,2,Australia,Pakistan,0.4,0,0,Brisbane,0,0,4,4,296,10
4,2,Australia,Pakistan,0.5,1,0,Brisbane,1,0,5,5,295,10
...,...,...,...,...,...,...,...,...,...,...,...,...,...
244,1795,Sri Lanka,Australia,39.5,0,9,Pallekele,194,39,5,239,61,1
245,1795,Sri Lanka,Australia,39.6,0,9,Pallekele,194,39,6,240,60,1
246,1795,Sri Lanka,Australia,39.7,1,9,Pallekele,195,39,7,241,59,1
247,1795,Sri Lanka,Australia,40.1,0,9,Pallekele,195,40,1,241,59,1


In [28]:
#calculate current run rate using current score and balls_bowled
df['crr'] = (df['current_score']*6)/df['balls_bowled']

C:\Users\User\AppData\Local\Temp\ipykernel_11140\2710979754.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['crr'] = (df['current_score']*6)/df['balls_bowled']


In [29]:
#calculate the score of last 10 overs. here we use rolling function. 1st divide the data frame into groups using match_id
#then run a loop. 
#window=60, cz last 10 overs 6*10=60 balls
groups = df.groupby('match_id')

match_ids = df['match_id'].unique()
last_ten = []
for id in match_ids:
    last_ten.extend(groups.get_group(id).rolling(window=60).sum()['runs'].values.tolist())

C:\Users\User\AppData\Local\Temp\ipykernel_11140\2855114552.py:9: FutureWarning: Dropping of nuisance columns in rolling operations is deprecated; in a future version this will raise TypeError. Select only valid columns before calling the operation. Dropped columns were Index(['batting_team', 'bowling_team', 'city'], dtype='object')
  last_ten.extend(groups.get_group(id).rolling(window=60).sum()['runs'].values.tolist())


In [30]:
last_ten #here 1 st 60 values are missing

[nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 56.0,
 60.0,
 60.0,
 61.0,
 62.0,
 61.0,
 61.0,
 61.0,
 61.0,
 60.0,
 61.0,
 62.0,
 62.0,
 62.0,
 61.0,
 62.0,
 61.0,
 60.0,
 61.0,
 61.0,
 62.0,
 62.0,
 62.0,
 58.0,
 59.0,
 59.0,
 58.0,
 57.0,
 57.0,
 57.0,
 57.0,
 58.0,
 58.0,
 57.0,
 56.0,
 56.0,
 55.0,
 56.0,
 56.0,
 56.0,
 57.0,
 56.0,
 54.0,
 55.0,
 56.0,
 50.0,
 50.0,
 49.0,
 50.0,
 46.0,
 47.0,
 46.0,
 46.0,
 43.0,
 39.0,
 35.0,
 36.0,
 36.0,
 36.0,
 36.0,
 33.0,
 30.0,
 32.0,
 32.0,
 32.0,
 33.0,
 33.0,
 33.0,
 34.0,
 34.0,
 35.0,
 34.0,
 34.0,
 38.0,
 38.0,
 38.0,
 40.0,
 40.0,
 39.0,
 40.0,
 39.0,
 40.0,
 39.0,
 43.0,
 41.0,
 42.0,
 42.0,
 43.0,
 43.0,
 43.0,
 44.0,
 47.0,
 4

In [31]:
df['last_ten'] = last_ten


C:\Users\User\AppData\Local\Temp\ipykernel_11140\2872724342.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['last_ten'] = last_ten


In [32]:
df

,match_id,batting_team,bowling_team,ball,runs,player_dismissed,city,current_score,over,ball_no,balls_bowled,balls_left,wickets_left,crr,last_ten
0,2,Australia,Pakistan,0.1,0,0,Brisbane,0,0,1,1,299,10,0.000000,NaN
1,2,Australia,Pakistan,0.2,0,0,Brisbane,0,0,2,2,298,10,0.000000,NaN
2,2,Australia,Pakistan,0.3,0,0,Brisbane,0,0,3,3,297,10,0.000000,NaN
3,2,Australia,Pakistan,0.4,0,0,Brisbane,0,0,4,4,296,10,0.000000,NaN
4,2,Australia,Pakistan,0.5,1,0,Brisbane,1,0,5,5,295,10,1.200000,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
244,1795,Sri Lanka,Australia,39.5,0,9,Pallekele,194,39,5,239,61,1,4.870293,53.0
245,1795,Sri Lanka,Australia,39.6,0,9,Pallekele,194,39,6,240,60,1,4.850000,49.0
246,1795,Sri Lanka,Australia,39.7,1,9,Pallekele,195,39,7,241,59,1,4.854772,50.0
247,1795,Sri Lanka,Australia,40.1,0,9,Pallekele,195,40,1,241,59,1,4.854772,50.0


In [33]:
#calculate final score
df.groupby('match_id').sum()['runs']

match_id
2       268
3       220
4       263
5       353
6       369
       ... 
1791    227
1792    288
1793    226
1794    212
1795    195
Name: runs, Length: 1398, dtype: int64

In [34]:
df.groupby('match_id').sum()['runs'].reset_index()

,match_id,runs
0,2,268
1,3,220
2,4,263
3,5,353
4,6,369
...,...,...
1393,1791,227
1394,1792,288
1395,1793,226
1396,1794,212


In [35]:
df.groupby('match_id').sum()['runs'].reset_index().merge(df,on='match_id')

,match_id,runs_x,batting_team,bowling_team,ball,runs_y,player_dismissed,city,current_score,over,ball_no,balls_bowled,balls_left,wickets_left,crr,last_ten
0,2,268,Australia,Pakistan,0.1,0,0,Brisbane,0,0,1,1,299,10,0.000000,NaN
1,2,268,Australia,Pakistan,0.2,0,0,Brisbane,0,0,2,2,298,10,0.000000,NaN
2,2,268,Australia,Pakistan,0.3,0,0,Brisbane,0,0,3,3,297,10,0.000000,NaN
3,2,268,Australia,Pakistan,0.4,0,0,Brisbane,0,0,4,4,296,10,0.000000,NaN
4,2,268,Australia,Pakistan,0.5,1,0,Brisbane,1,0,5,5,295,10,1.200000,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
405788,1795,195,Sri Lanka,Australia,39.5,0,9,Pallekele,194,39,5,239,61,1,4.870293,53.0
405789,1795,195,Sri Lanka,Australia,39.6,0,9,Pallekele,194,39,6,240,60,1,4.850000,49.0
405790,1795,195,Sri Lanka,Australia,39.7,1,9,Pallekele,195,39,7,241,59,1,4.854772,50.0
405791,1795,195,Sri Lanka,Australia,40.1,0,9,Pallekele,195,40,1,241,59,1,4.854772,50.0


In [36]:
final_df1 = df.groupby('match_id').sum()['runs'].reset_index().merge(df,on='match_id')

In [37]:
final_df1=final_df1[['batting_team','bowling_team','city','current_score','balls_left','wickets_left','crr','last_ten','runs_x']]

In [38]:
final_df1

,batting_team,bowling_team,city,current_score,balls_left,wickets_left,crr,last_ten,runs_x
0,Australia,Pakistan,Brisbane,0,299,10,0.000000,NaN,268
1,Australia,Pakistan,Brisbane,0,298,10,0.000000,NaN,268
2,Australia,Pakistan,Brisbane,0,297,10,0.000000,NaN,268
3,Australia,Pakistan,Brisbane,0,296,10,0.000000,NaN,268
4,Australia,Pakistan,Brisbane,1,295,10,1.200000,NaN,268
...,...,...,...,...,...,...,...,...,...
405788,Sri Lanka,Australia,Pallekele,194,61,1,4.870293,53.0,195
405789,Sri Lanka,Australia,Pallekele,194,60,1,4.850000,49.0,195
405790,Sri Lanka,Australia,Pallekele,195,59,1,4.854772,50.0,195
405791,Sri Lanka,Australia,Pallekele,195,59,1,4.854772,50.0,195


In [39]:
#then drop the missing values
final_df1.isnull().sum()

batting_team         0
bowling_team         0
city                 0
current_score        0
balls_left           0
wickets_left         0
crr                  0
last_ten         82191
runs_x               0
dtype: int64

In [40]:
#as 82191 values are missing
final_df1.dropna(inplace=True)

In [41]:
final_df1

,batting_team,bowling_team,city,current_score,balls_left,wickets_left,crr,last_ten,runs_x
59,Australia,Pakistan,Brisbane,56,243,7,5.894737,56.0,268
60,Australia,Pakistan,Brisbane,60,242,7,6.206897,60.0,268
61,Australia,Pakistan,Brisbane,60,241,7,6.101695,60.0,268
62,Australia,Pakistan,Brisbane,61,240,7,6.100000,61.0,268
63,Australia,Pakistan,Brisbane,62,239,7,6.098361,62.0,268
...,...,...,...,...,...,...,...,...,...
405788,Sri Lanka,Australia,Pallekele,194,61,1,4.870293,53.0,195
405789,Sri Lanka,Australia,Pallekele,194,60,1,4.850000,49.0,195
405790,Sri Lanka,Australia,Pallekele,195,59,1,4.854772,50.0,195
405791,Sri Lanka,Australia,Pallekele,195,59,1,4.854772,50.0,195


In [42]:
#for shuffling
final_df1 = final_df1.sample(final_df1.shape[0])

In [43]:
final_df1

,batting_team,bowling_team,city,current_score,balls_left,wickets_left,crr,last_ten,runs_x
166791,South Africa,Zimbabwe,Harare,166,126,8,5.724138,65.0,323
259996,Sri Lanka,Australia,Hambantota,55,218,8,4.024390,37.0,208
357013,Pakistan,New Zealand,Sharjah,72,230,9,6.171429,63.0,364
209458,Australia,India,Delhi,226,2,5,4.550336,61.0,229
45852,Sri Lanka,England,Colombo,158,162,9,6.869565,63.0,366
...,...,...,...,...,...,...,...,...,...
269776,Pakistan,Zimbabwe,Bulawayo,108,171,8,5.023256,62.0,247
100205,Sri Lanka,England,Manchester,192,111,7,6.095238,52.0,318
113239,Pakistan,Sri Lanka,Colombo,39,220,8,2.925000,28.0,201
71414,West Indies,Bangladesh,Dhaka,32,237,9,3.047619,32.0,195


In [44]:
final_df1.sample(2)

,batting_team,bowling_team,city,current_score,balls_left,wickets_left,crr,last_ten,runs_x
134545,New Zealand,Sri Lanka,Mumbai,61,186,7,3.210526,36.0,165
57703,India,Afghanistan,Southampton,64,216,9,4.571429,57.0,224


In [45]:
X = final_df1.drop(columns=['runs_x'])
y = final_df1['runs_x']
from sklearn.model_selection import train_test_split
X_train,X_test,y_train,y_test = train_test_split(X,y,test_size=0.2,random_state=1)

In [46]:
X_train

,batting_team,bowling_team,city,current_score,balls_left,wickets_left,crr,last_ten
295251,Australia,Sri Lanka,Melbourne,199,75,8,5.306667,60.0
318579,India,Bangladesh,Dhaka,175,120,8,5.833333,42.0
134933,England,Australia,Jaipur,127,107,5,3.948187,29.0
96424,Sri Lanka,India,Pune,144,131,7,5.112426,47.0
108512,New Zealand,West Indies,Wellington,199,84,9,5.527778,58.0
...,...,...,...,...,...,...,...,...
260936,Australia,Sri Lanka,Colombo,131,131,7,4.650888,48.0
267226,England,India,Delhi,209,47,4,4.956522,59.0
23558,Afghanistan,Zimbabwe,Harare,106,175,7,5.088000,39.0
230526,India,Sri Lanka,Mirpur,195,59,4,4.854772,46.0


In [47]:
from sklearn.compose import ColumnTransformer
from sklearn.preprocessing import OneHotEncoder
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import StandardScaler
from sklearn.ensemble import RandomForestRegressor
from xgboost import XGBRegressor
from sklearn.metrics import r2_score,mean_absolute_error

In [48]:
trf1 = ColumnTransformer([
    ('trf1',OneHotEncoder(sparse=False,drop='first'),['batting_team','bowling_team','city'])
]
,remainder='passthrough')

In [50]:
pipe_ODI = Pipeline(steps=[
    ('step1',trf1),
    ('step2',StandardScaler()),
    ('step3',XGBRegressor(n_estimators=1000,learning_rate=0.2,max_depth=12,random_state=1))
])

In [51]:
pipe_ODI.fit(X_train,y_train)
y_pred = pipe_ODI.predict(X_test)
print(r2_score(y_test,y_pred))
print(mean_absolute_error(y_test,y_pred))

0.9932583139550756
2.2427677577086476


In [52]:
#export it
#f = open("pipe_ODI.pkl", mode="w", encoding="utf-8")
pickle.dump(pipe_ODI,open('pipe_ODI.pkl','wb'))

In [53]:
import xgboost
xgboost.__version__

'1.6.0'

In [54]:
#there are 114 cities, but we will not consider all. we will consider where minimum 5 matches occur.
df['city'].value_counts()[df['city'].value_counts() > 1500].index.tolist()

['Mirpur',
 'London',
 'Colombo',
 'Harare',
 'Bulawayo',
 'Sydney',
 'Abu Dhabi',
 'Rangiri',
 'Melbourne',
 'Centurion',
 'Sharjah',
 'Adelaide',
 'Perth',
 'Birmingham',
 'Dubai',
 'Auckland',
 'Johannesburg',
 'Brisbane',
 'Pallekele',
 'Wellington',
 'Cardiff',
 'Durban',
 'Hamilton',
 'Port Elizabeth',
 'Manchester',
 'Nottingham',
 'Southampton',
 'Antigua',
 'Cape Town',
 'Guyana',
 'Trinidad',
 'Karachi',
 'Christchurch',
 'Hambantota',
 'Leeds',
 'Chandigarh',
 'Napier',
 'Jamaica',
 'St Lucia',
 'Lahore',
 'Mumbai',
 'St Kitts',
 'Chester-le-Street',
 'Chittagong',
 'Ahmedabad',
 'Barbados',
 'Dhaka',
 'Grenada',
 'Hobart',
 'Jaipur',
 'Delhi',
 'Mount Maunganui',
 'Nagpur',
 'Visakhapatnam',
 'Chennai',
 'Bloemfontein',
 'Dunedin',
 'Fatullah',
 'St Vincent',
 'Bristol',
 'Nelson',
 'Canberra',
 'Rajkot',
 'Kolkata',
 'Hyderabad',
 'Kanpur',
 'Potchefstroom',
 'Cuttack',
 'Kuala Lumpur',
 'Indore',
 'Pune',
 'Rawalpindi',
 'Paarl',
 'Multan']